![](https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png)
# Production coding: Logging
Week 11 | Day 1


### LEARNING OBJECTIVES
*After this lesson, you will be able to:*
- Explain the benefits of logging
- Use the Python logger library
- Configure Python logger

## Data Scientist vs. Machine Learning Engineer

<img src="http://i.imgur.com/S3q5GJn.png" width=400>

This is a distinction that is becoming more relevant recently. While both roles must understand the data science workflow and the ins and outs of the various machine learning algorithms, typically the output is different. The machine learning engineer's final product is a working model in production code, while the data scientist's is typically a number of visualizations and/or a prototype.

Regardless of whether the code is for a prototype or for production, logging is an essential component.

## Logging

Why log?

## Because this is not ok

```python
print "About to run this questionable function that keeps breaking"

do_someting_sketchy()

print "Yes, made it!!!"

```


## Ad hoc `print` statements: 
- Not crazy for simple personal scripts. Not sufficient for anything more

## Grown-up coding

- Writing to stderr or stdout doesn't cut it for long-running scripts
- For more mature projects, we need a more robust system of to diagnose issues and keep an audit trail of events

## Python's `logging` library

- Python's standard library includes ``logging``
- It is a versatile library that includes a great deal of functionality

## Code-a-long: Flask app

- Create a folder on your desktop called `my_flask_api`
- Create a text file with the following content:
    ```python
    from flask import Flask
    app = Flask(__name__)

    @app.route('/')
    def hello_world():
        print 'about to print this!'
        return 'Hello, World!'

    if __name__ == '__main__':
        print 'about to run this thing!'
        app.run()
    ```
- Save the file as `test_flask.py`
- On your terminal, cd into the `my_flask_api` directory and type: `python test_flask.py`
- Pull up your browser and notice the output of the print statements

## Codealong: Flask app


- Let's now improve it a bit; we'll write our output to a file
- Better, right?
- But notice that this file is not readable while the application is running
- Also, imagine how this would this scale to dozens, hundreds, or thousands of users? How about as our code became increasingly complex?

## A better way: the `logging` module


We'll now add logging statements that output a .log file. (control + c to kill your existing flask instance)

```python
from flask import Flask
import logging
logging.basicConfig(filename='test.log', level=logging.INFO)
logger = logging.getLogger(__name__)
app = Flask(__name__)

@app.route('/')
def hello_world():
    logger.info('about to print this!')
    return 'Hello, World!'

if __name__ == '__main__':
    logger.debug('about to run this thing!')
    app.run()
```

This configures our logging system (specifically, the "root logger"), by specifying a write file and a "level" (more on that in a moment).


## The `logging` module

- Run this code again with `python test_flask.py`
- Refresh your browser and check the `test.log` output, is anything missing? Why might that be?

## The logging hierarchy

In our code, notice this line:
```python
logging.basicConfig(filename='test.log', level=logging.INFO)
```
This line configured our output so that only things at the level of INFO or more critical are written to the log.
Go ahead and rerun the process with `level=logging.DEBUG`. What do you see now?

## The logging hierarchy

There are five "levels" of logging. You  configure the system's threshold for logging via logging.basicConfig.

```python
logger.debug('Very granular logging message, useful for debugging.')
logger.info('Simple update on normal execution, e.g. "Processing record {} of {}"'.format(10,100)
logger.warning('You\'ve seen these in sklearn, warning about methods being deprecated')
logger.error('Logs an error message')
logger.critical('Well this is an issue')
```

There are other useful configuration parameters, particularly format:

```python
logging.basicConfig(filename='test.log', format='%(levelname)s: %(asctime)s: %(message)s', level=logging.DEBUG)
```

You can also change it so that the log file will be over-written every time the program is run:

```python
logging.basicConfig(filename='test.log', filemode='w', level=logging.DEBUG)
```

NB: You'll need to add this at the start of your file, as .basicConfig does nothing if it's run after your logging system is configured.

## Beyond the basics: Dealing with scale

One thing you can imagine might happen is A LOT of text being written to the log files, especially as you scale up your application. How can you handle that?

## Rotating log files

One way to deal with this issue is to write to a file until it reaches a certain max size. Once that happens, you begin writing to another file. Generally, you only want to save a certain number of files so that you don't clutter up your system.

[RotatingFileHandler](https://docs.python.org/3/library/logging.handlers.html#logging.handlers.RotatingFileHandler)

## Timed rotating log files

Another way to handle log files is to rotate them on a given schedule. For example, you could write all the data for each hour into a log file and retain only the last 24 hours or the last 72 hours.

[TimedRotatingFileHandler](https://docs.python.org/3/library/logging.handlers.html#logging.handlers.TimedRotatingFileHandler)

## Example: Try out the following

```python
from flask import Flask
import logging
from logging.handlers import TimedRotatingFileHandler

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

fh = TimedRotatingFileHandler('test.log', when='s', interval=5, backupCount=5)
fh.setLevel(logging.DEBUG)
fh.setFormatter(formatter)
logger.addHandler(fh)

app = Flask(__name__)

@app.route('/')
def hello_world():
    logger.info('about to print this!')
    return 'Hello, World!'

if __name__ == '__main__':
    logger.debug('about to run this thing!')
    app.run()
```

## Additional FileHandlers

This file handler can send email messages:
[SMTPFileHandler](https://docs.python.org/3/library/logging.handlers.html#logging.handlers.SMTPHandler)<br>

This one can send a http request:
[HTTPFileHandler](https://docs.python.org/3/library/logging.handlers.html#logging.handlers.HTTPHandler)

Note that you can have multiple handlers with different levels. For example, maybe INFO and WARNINGS go to a log file, but CRITICAL goes to your email.

### Independent practice: logging

Using the following starter script, your new logging knowledge, and the Postman Chrome add in, continue developing a Flask API. Try using various handlers like SMTPFileHandler. Try using additional arguments, as well as using POST and GET.

The following resources should help: <br>
[Simple Flask API](http://blog.luisrei.com/articles/flaskrest.html) <br>
[RESTful Flask API](https://blog.miguelgrinberg.com/post/designing-a-restful-api-with-python-and-flask) <br>
[Logging Cookbook](https://docs.python.org/2/howto/logging-cookbook.html)<br>


```python
from flask import Flask
import logging
from logging.handlers import TimedRotatingFileHandler
from flask import request

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

fh = TimedRotatingFileHandler('test.log', when='m', interval=5, backupCount=3)
fh.setLevel(logging.DEBUG)
fh.setFormatter(formatter)
logger.addHandler(fh)

app = Flask(__name__)

@app.route('/')
def hello_world():
    logger.info('about to print this!')
    return 'Hello, World!'

@app.route('/hello')
def say_hi():
    if 'name' in request.args:
	    return 'Hello, ' + request.args['name']
    else:
	    return 'Hello, John Doe'

if __name__ == '__main__':
    logger.debug('about to run this thing!')
    app.run()
```

## Postman add-in

<img src="http://i.imgur.com/MHl4UbS.png">

## Conclusion

In this lecture we saw the following:
- Why to log
- When to log
- How to log